In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import csv


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
# import training data
from sklearn import preprocessing


def import_training_labels():
    labels = []
    with open('train.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            labels.append(row[1])

    labels.pop(0)

    return labels

label_list = ["blues", "hiphop", "jazz", "metal", "disco", "country", "rock", "reggae", "classical", "pop"]


training_data = np.load("Training_Data_vector.npy")

means = np.mean(training_data, axis=1)
means = means.reshape(len(means), 1)
std_devs = np.std(training_data, axis=1)
std_devs = std_devs.reshape(len(std_devs), 1)

norm_training_data = (training_data - (means * np.ones((1, 700)))) / (std_devs * np.ones((1, 700)))
tensor_train_data = torch.from_numpy(norm_training_data)


training_labels = import_training_labels()
le = preprocessing.LabelEncoder() # Changes text labels to numerical tensors
le.fit(label_list)
new_labels = le.transform(training_labels)
tensor_labels = torch.tensor(new_labels)
tensor_labels.resize(len(new_labels), 1) # TODO: fix this method so it's not the deprecated one


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


entries_per_sample = 24


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.full_connect1 = nn.Linear(entries_per_sample, 256)
        self.full_connect2 = nn.Linear(256, 128)
        self.full_connect3 = nn.Linear(128, 64)
        self.full_connect4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.full_connect1(x))
        x = F.relu(self.full_connect2(x))
        x = F.relu(self.full_connect3(x))
        x = F.log_softmax(self.full_connect4(x), dim=0)
#         x = self.full_connect4(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(150):  # loop over the dataset multiple times
    
    running_loss = 0.0 #reset every epoch
    for i in range(len(tensor_labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs = tensor_train_data[:,i]
        label = tensor_labels[i]
        label = label.reshape((1,))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float()).reshape(1, 10)
    
        loss = criterion(outputs, label)
#         loss = F.nll_loss(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    if epoch % 10 == 0:
            print(f"Finished epoch {epoch}, loss = {running_loss}")

print(f'Finished Training, final loss = {running_loss}')

In [ ]:
# RUN TO EXPORT CURRENT MODEL
PATH = './nn_classfier1.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# RUN TO LOAD SAVED MODEL
net = Net()
PATH = './nn_classifier1.pth'
net.load_state_dict(torch.load(PATH))

In [ ]:
# Evaluate Training Data
training_outputs = net(tensor_train_data.T.float())
_, predicted_train = torch.max(training_outputs, 1)

predicted_training_labels = le.inverse_transform(predicted_train)
# print(predicted_training_labels)

accuracy = torch.sum(predicted_train == tensor_labels)/700
print(f"Accuracy on Training Data: {accuracy}")

In [ ]:
# Evaluate Test Data

test_data_vector = np.load("Test_Data_vector.npy")
norm_test_data = (test_data_vector- (means * np.ones((1, 300)))) / (std_devs * np.ones((1, 300)))
test_tensor = torch.from_numpy(norm_test_data)

# print(test_tensor)
outputs = net(test_tensor.T.float())
# print(torch.max(net(tensor_train_data.T.float()), 1))
# print(outputs)
_, predicted = torch.max(outputs, 1)

predicted_labels = le.inverse_transform(predicted)
print(predicted_labels)



In [ ]:
# Export Test Data

file_name = "pytorch_classifier2"

with open(f'{file_name}.csv', 'w', newline='') as csvfile:
    wrtr = csv.writer(csvfile, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    wrtr.writerow(["filename", "label"])
    output_lists = [[f"sample{i+700}.wav", predicted_labels[i]] for i in range(300)]
    wrtr.writerows(output_lists)
print(f"{file_name} exported successfully")
